In [6]:
# Cell 1: Setup and Imports
import re

# Note: Uncomment and install these libraries in a real environment
# ! pip install openai pdfplumber
import openai
import pdfplumber

# Simulated OpenAI API key setup (replace with real key in practice)
openai.api_key = "your_api key"

# Define section mapping for identifying sections
section_mapping = {
    "problem": ["problem", "pain point", "challenge"],
    "solution": ["solution", "product", "service"],
    "market": ["market", "market opportunity", "target market"],
    "business_model": ["business model", "revenue model", "monetization"],
    "financials": ["financials", "projections", "financial plan"],
    "team": ["team", "founders", "management"]
}

# Invert mapping for easier lookup
heading_to_section = {}
for section, headings in section_mapping.items():
    for heading in headings:
        heading_to_section[heading.lower()] = section

# Define criteria for evaluating each section
section_criteria = {
    "problem": "Clearly defines the pain point, shows market need, is specific and relatable.",
    "solution": "Clearly addresses the problem, is innovative, feasible, and scalable.",
    "market": "Defines the target market, shows market size, growth potential, and competition.",
    "business_model": "Explains how the startup will make money, pricing strategy, sales channels.",
    "financials": "Provides realistic projections, shows understanding of costs and revenues.",
    "team": "Highlights relevant experience, skills, and why the team is capable of executing the plan."
}

# Define weights for each section (summing to 1.0)
section_weights = {
    "problem": 0.2,
    "solution": 0.2,
    "market": 0.15,
    "business_model": 0.15,
    "financials": 0.15,
    "team": 0.15
}

In [7]:
# Cell 2: Text Extraction Function (for PDFs)
def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF pitch deck using pdfplumber.
    Note: This is a placeholder; actual PDFs not used in demo.
    """
    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return ""

In [8]:
# Cell 3: Section Extraction Function
def extract_sections(text):
    """
    Extract key sections from pitch deck text based on headings.
    """
    sections = {}
    current_section = None
    for line in text.split('\n'):
        line = line.strip()
        if line.lower() in heading_to_section:
            current_section = heading_to_section[line.lower()]
            sections[current_section] = ""
        elif current_section:
            sections[current_section] += line + " "
    return sections

In [9]:
# Cell 4: Simulated LLM Evaluation Function

def parse_response(response_text):
    """
    Parse LLM response into score, strengths, weaknesses, and suggestions.
    """
    lines = response_text.strip().split('\n')
    score = None
    strengths = ""
    weaknesses = ""
    suggestions = ""
    current_part = None

    for line in lines:
        if line.startswith("Score:"):
            score = float(line.split(":")[1].strip().split("/")[0])
            current_part = None
        elif line.startswith("Strengths:"):
            current_part = "strengths"
        elif line.startswith("Weaknesses:"):
            current_part = "weaknesses"
        elif line.startswith("Suggestions:"):
            current_part = "suggestions"
        elif current_part:
            if current_part == "strengths":
                strengths += line + " "
            elif current_part == "weaknesses":
                weaknesses += line + " "
            elif current_part == "suggestions":
                suggestions += line + " "
    return score, strengths.strip(), weaknesses.strip(), suggestions.strip()

def evaluate_section(section_name, section_text):
    """
    Simulate LLM evaluation of a section (replace with actual API call in practice).
    Returns score, strengths, weaknesses, and suggestions.
    """
    # Simulated prompt (in practice, send this to OpenAI API)
    prompt = f"""
        Evaluate the following {section_name} section from a startup pitch deck.
        Provide a score out of 10 based on how well it addresses: {section_criteria[section_name]}
        Also, provide strengths, weaknesses, and suggestions.

        Structure your response as:
        Score: X/10
        Strengths: ...
        Weaknesses: ...
        Suggestions: ...

        Section text: {section_text}
        """

    # Mock response (replace with actual API call)
    # Example: response = openai.ChatCompletion.create(model="gpt-4", messages=[...])
    if section_name == "problem":
        response_text = """
            Score: 8/10
            Strengths: Clearly identifies a relatable pain point in the freelance market.
            Weaknesses: Lacks specific data to quantify the market need.
            Suggestions: Include statistics or examples to strengthen the case.
            """
    elif section_name == "solution":
        response_text = """
            Score: 7/10
            Strengths: Offers a clear solution with useful features like messaging.
            Weaknesses: Scalability not addressed.
            Suggestions: Explain how the platform scales with more users.
            """
    else:
        response_text = f"""
            Score: 6/10
            Strengths: Provides basic information for {section_name}.
            Weaknesses: Lacks depth and specificity.
            Suggestions: Add more detailed data or examples.
            """

    # Parse the response
    return parse_response(response_text)

In [10]:
# Cell 5: Response Parsing Function
def parse_response(response_text):
    """
    Parse LLM response into score, strengths, weaknesses, and suggestions.
    """
    lines = response_text.strip().split('\n')
    score = None
    strengths = ""
    weaknesses = ""
    suggestions = ""
    current_part = None

    for line in lines:
        if line.startswith("Score:"):
            score = float(line.split(":")[1].strip().split("/")[0])
            current_part = None
        elif line.startswith("Strengths:"):
            current_part = "strengths"
        elif line.startswith("Weaknesses:"):
            current_part = "weaknesses"
        elif line.startswith("Suggestions:"):
            current_part = "suggestions"
        elif current_part:
            if current_part == "strengths":
                strengths += line + " "
            elif current_part == "weaknesses":
                weaknesses += line + " "
            elif current_part == "suggestions":
                suggestions += line + " "
    return score, strengths.strip(), weaknesses.strip(), suggestions.strip()

In [11]:
# Cell 6: Calculate Overall Score
def calculate_overall_score(section_scores):
    """
    Calculate weighted overall pitch score out of 100.
    """
    weighted_sum = sum((score / 10) * section_weights[section] 
                      for section, score in section_scores.items())
    return weighted_sum * 100

In [12]:
# Cell 7: Sample Pitch Deck Analysis
# Sample Pitch Deck 1
pitch_deck_1 = """
Problem
The current market lacks an efficient way to connect freelancers with clients, leading to missed opportunities.
Solution
Our platform provides a seamless interface for freelancers and clients to connect, with real-time messaging.
Market
The freelance market is growing rapidly, with an estimated size of $1.5 trillion by 2025.
Business Model
We take a 10% commission on each transaction.
Financials
Projected revenue of $10 million in year one.
Team
Experienced entrepreneurs and developers with successful startups.
"""

# Sample Pitch Deck 2
pitch_deck_2 = """
Problem
Urban commuters waste time due to inefficient public transport.
Solution
A mobile app optimizes routes using real-time traffic data.
Market
Targeting a $500 million urban mobility market.
Business Model
Subscription model at $5/month per user.
Financials
Break-even in 18 months with 100k users.
Team
Team with logistics and tech expertise.
"""

# Sample Pitch Deck 3
pitch_deck_3 = """
Problem
Small businesses struggle with affordable marketing.
Solution
An AI tool automates social media campaigns.
Market
$200 million market of small business owners.
Business Model
Freemium model with premium features at $20/month.
Financials
$5 million revenue projected in year two.
Team
Marketing and AI specialists.
"""

# Analyze all pitch decks
pitch_decks = [pitch_deck_1, pitch_deck_2, pitch_deck_3]
for i, pitch_deck_text in enumerate(pitch_decks, 1):
    print(f"\n=== Pitch Deck {i} Analysis ===\n")
    
    # Extract sections
    sections = extract_sections(pitch_deck_text)
    
    # Evaluate each section
    section_scores = {}
    feedback = {}
    for section in sections:
        score, strengths, weaknesses, suggestions = evaluate_section(section, sections[section])
        section_scores[section] = score
        feedback[section] = {
            "strengths": strengths,
            "weaknesses": weaknesses,
            "suggestions": suggestions
        }
    
    # Calculate overall score
    overall_score = calculate_overall_score(section_scores)
    
    # Display results
    print(f"Overall Pitch Score: {overall_score:.2f}/100")
    for section in feedback:
        print(f"\nSection: {section.capitalize()}")
        print(f"Score: {section_scores[section]}/10")
        print(f"Strengths: {feedback[section]['strengths']}")
        print(f"Weaknesses: {feedback[section]['weaknesses']}")
        print(f"Suggestions: {feedback[section]['suggestions']}")


=== Pitch Deck 1 Analysis ===

Overall Pitch Score: 66.00/100

Section: Problem
Score: 8.0/10
Strengths: 
Weaknesses: 
Suggestions: 

Section: Solution
Score: 7.0/10
Strengths: 
Weaknesses: 
Suggestions: 

Section: Market
Score: 6.0/10
Strengths: 
Weaknesses: 
Suggestions: 

Section: Business_model
Score: 6.0/10
Strengths: 
Weaknesses: 
Suggestions: 

Section: Financials
Score: 6.0/10
Strengths: 
Weaknesses: 
Suggestions: 

Section: Team
Score: 6.0/10
Strengths: 
Weaknesses: 
Suggestions: 

=== Pitch Deck 2 Analysis ===

Overall Pitch Score: 66.00/100

Section: Problem
Score: 8.0/10
Strengths: 
Weaknesses: 
Suggestions: 

Section: Solution
Score: 7.0/10
Strengths: 
Weaknesses: 
Suggestions: 

Section: Market
Score: 6.0/10
Strengths: 
Weaknesses: 
Suggestions: 

Section: Business_model
Score: 6.0/10
Strengths: 
Weaknesses: 
Suggestions: 

Section: Financials
Score: 6.0/10
Strengths: 
Weaknesses: 
Suggestions: 

Section: Team
Score: 6.0/10
Strengths: 
Weaknesses: 
Suggestions: 

=== Pit